In [ ]:
import os
import cv2
import numpy as np
import random

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def parse_label(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()

    annotations = []
    for line in lines:
        parts = line.strip().split()
        class_prob = float(parts[0])  # Probabilidad o confianza de la clase
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])

        # Corregir coordenadas si están fuera del rango [0, 1]
        x_min = max(0, x_center - width / 2)
        y_min = max(0, y_center - height / 2)
        x_max = min(1, x_center + width / 2)
        y_max = min(1, y_center + height / 2)

        annotations.append([class_prob, x_min, y_min, x_max, y_max])

    return np.array(annotations)

In [ ]:
def random_brightness(image, factor_range=(-20, 20)):
    # Ajustar el brillo aleatoriamente
    factor = random.randint(factor_range[0], factor_range[1])
    return np.clip(image.astype(int) + factor, 0, 255).astype(np.uint8)

def random_contrast(image, factor_range=(0.8, 1.2)):
    # Ajustar el contraste aleatoriamente
    factor = random.uniform(factor_range[0], factor_range[1])
    mean = np.mean(image, axis=(0, 1), keepdims=True)
    return np.clip((image.astype(float) - mean) * factor + mean, 0, 255).astype(np.uint8)

def random_saturation(image, factor_range=(0.1, 1.2)):
    # Ajustar la saturación aleatoriamente
    factor = random.uniform(factor_range[0], factor_range[1])
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[..., 1] = np.clip(hsv[..., 1] * factor, 0, 255).astype(np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

In [ ]:
def augment_images(images_dir, labels_dir, output_images_dir, output_labels_dir):
    os.makedirs(output_images_dir, exist_ok=True)
    os.makedirs(output_labels_dir, exist_ok=True)

    image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]
    for image_file in image_files:
        image_path = os.path.join(images_dir, image_file)
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(labels_dir, label_file)

        if os.path.exists(label_path):
            # Cargar la imagen sin redimensionar
            image = cv2.imread(image_path)

            # Verificar si la imagen es a color
            if len(image.shape) < 3 or image.shape[2] < 3:
                continue  # Saltar imágenes en escala de grises o con canales insuficientes

            original_height, original_width = image.shape[:2]  # Obtener dimensiones originales

            # Aplicar ajustes de brillo, contraste y saturación de manera aleatoria
            image = random_brightness(image)
            image = random_contrast(image)
            image = random_saturation(image)

            # Ajustar la imagen de nuevo a sus dimensiones originales si cambió
            if image.shape[:2] != (original_height, original_width):
                image = cv2.resize(image, (original_width, original_height))

            # Cargar las etiquetas
            annotations = parse_label(label_path)

            # Guardar la imagen original sin cambios
            output_image_path = os.path.join(output_images_dir, image_file)
            cv2.imwrite(output_image_path, image)

            # Guardar las etiquetas
            output_label_path = os.path.join(output_labels_dir, label_file)
            with open(output_label_path, 'w') as f:
                for annotation in annotations:
                    f.write(' '.join([str(x) for x in annotation]) + '\n')
        else:
            print(f"Etiqueta no encontrada para la imagen: {image_file}")

In [ ]:
# Directorios de entrada y salida para datos de entrenamiento y prueba
train_images_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Train/images'
train_labels_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Train/labels'
test_images_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Test/images'
test_labels_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Test/labels'

# Directorios de salida para imágenes y etiquetas aumentadas
train_aug_images_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Train/imagesAugmented'
train_aug_labels_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Train/labelsAugmented'
test_aug_images_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Test/imagesAugmented'
test_aug_labels_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Test/labelsAugmented'

# Aplicar aumento de datos a las imágenes de entrenamiento y prueba
augment_images(train_images_dir, train_labels_dir, train_aug_images_dir, train_aug_labels_dir)
augment_images(test_images_dir, test_labels_dir, test_aug_images_dir, test_aug_labels_dir)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Test/images'

In [ ]:
import os
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt

def load_images(image_dir, max_width):
    images = []
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        image = cv2.imread(image_path)
        if image.shape[1] <= max_width:
            images.append(image)
    return images

original_images_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Train/images'
augmented_images_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Train/imagesAugmented'

max_image_width = 1000

original_images = load_images(original_images_dir, max_image_width)
augmented_images = load_images(augmented_images_dir, max_image_width)

random.seed(131313)
sample_indices = random.sample(range(len(original_images)), 10)

# Crear la visualización de las imágenes antes y después del data augmentation
num_rows = 8  # Número de filas que deseas mostrar
num_cols = 2  # Siempre será 2 (una columna para "Original" y otra para "Augmented")
fig, axs = plt.subplots(num_rows, num_cols, figsize=(12, 20))

idx = 0
for i in range(num_rows):
    if idx < len(sample_indices):
        # Mostrar imagen original
        axs[i, 0].imshow(cv2.cvtColor(original_images[sample_indices[idx]], cv2.COLOR_BGR2RGB))
        axs[i, 0].set_title('Original')
        axs[i, 0].axis('off')

        # Mostrar imagen aumentada
        axs[i, 1].imshow(cv2.cvtColor(augmented_images[sample_indices[idx]], cv2.COLOR_BGR2RGB))
        axs[i, 1].set_title('Augmented')
        axs[i, 1].axis('off')
        idx += 1
    else:
        # Si se alcanza el número máximo de imágenes disponibles, ocultar la subparcela
        axs[i, 0].axis('off')
        axs[i, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
images_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Train/images'

valid_images = []
image_files = os.listdir(images_dir)

while len(valid_images) < 3:
    # Elegir una imagen aleatoria
    image_file = random.choice(image_files)
    image_path = os.path.join(images_dir, image_file)
    image = cv2.imread(image_path)

    # Verificar el ancho de la imagen
    if image is not None and image.shape[1] <= 1000:
        valid_images.append(image)

# Crear una figura con 3 filas y 3 columnas para mostrar las imágenes
fig, axes = plt.subplots(3, 3, figsize=(12, 12))

for i in range(3):
    # Seleccionar una imagen aleatoria de la lista de imágenes válidas
    image = valid_images[i]

    # Aplicar transformaciones de brillo, contraste y saturación
    transformed_brightness_min = random_brightness(image, factor_range=(-20, -20))
    transformed_brightness_max = random_brightness(image, factor_range=(20, 20))
    transformed_contrast_min = random_contrast(image, factor_range=(0.8, 0.8))
    transformed_contrast_max = random_contrast(image, factor_range=(1.2, 1.2))
    transformed_saturation_min = random_saturation(image, factor_range=(0.1, 0.1))
    transformed_saturation_max = random_saturation(image, factor_range=(1.2, 1.2))

    # Mostrar imágenes en la cuadrícula organizadas por tipo de transformación
    axes[0, i].imshow(cv2.cvtColor(transformed_brightness_min, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('Original')
    axes[1, 0].set_title('Original')
    axes[2, 0].set_title('Original')
    axes[0, 1].set_title('Brillo Min')
    axes[1, 1].set_title('Contraste Min')
    axes[2, 1].set_title('Saturacion Min')
    axes[0, 2].set_title('Brillo Max')
    axes[1, 2].set_title('Contraste Max')
    axes[2, 2].set_title('Saturacion Max')
    axes[1, i].imshow(cv2.cvtColor(transformed_contrast_min, cv2.COLOR_BGR2RGB))
    axes[2, i].imshow(cv2.cvtColor(transformed_saturation_min, cv2.COLOR_BGR2RGB))

    # Eliminar espacios entre las imágenes
    for ax in axes[:, i]:
        ax.axis('off')

# Mostrar la cuadrícula de imágenes
plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import os

# Directorio donde se encuentran las imágenes
images_dir = '/content/drive/MyDrive/ITBA - Analitica y +/5.1 Proyecto Final 82.02/Carpeta IMP/Train_Test/Train/images'

def load_image_by_name(image_name):
    # Construir la ruta completa de la imagen
    image_path = os.path.join(images_dir, image_name)

    # Cargar la imagen si existe
    if os.path.isfile(image_path):
        return cv2.imread(image_path)
    else:
        print(f"Imagen '{image_name}' no encontrada en el directorio '{images_dir}'")
        return None

def random_brightness(image, factor_range=(-20, 20)):
    # Ajustar el brillo aleatoriamente
    factor = random.randint(factor_range[0], factor_range[1])
    return np.clip(image.astype(int) + factor, 0, 255).astype(np.uint8)

def random_contrast(image, factor_range=(0.8, 1.2)):
    # Ajustar el contraste aleatoriamente
    factor = random.uniform(factor_range[0], factor_range[1])
    mean = np.mean(image, axis=(0, 1), keepdims=True)
    return np.clip((image.astype(float) - mean) * factor + mean, 0, 255).astype(np.uint8)

def random_saturation(image, factor_range=(0.1, 1.2)):
    # Ajustar la saturación aleatoriamente
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    factor = random.uniform(factor_range[0], factor_range[1])
    hsv[..., 1] = np.clip(hsv[..., 1] * factor, 0, 255).astype(np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def plot_images_with_transforms(image, title_original, transform_functions, titles_transforms, figsize=(12, 4)):
    # Crear una figura con subgráficos para cada transformación
    fig, axes = plt.subplots(1, len(transform_functions) + 1, figsize=figsize)
    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0].set_title(title_original)
    axes[0].axis('off')

    for i, (transform_func, title) in enumerate(zip(transform_functions, titles_transforms), start=1):
        transformed_image = transform_func(image)
        axes[i].imshow(cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB))
        axes[i].set_title(title)
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Cargar imágenes específicas por nombre
image1 = load_image_by_name('3227_40-11-1-4.png')
image2 = load_image_by_name('53821_2-7-1-3.png')
image3 = load_image_by_name('53750_1-1-10-4.png')

# Definir funciones de transformación y títulos
transform_functions_brightness = [
    lambda x: x,  # Función de identidad para la imagen original
    lambda x: random_brightness(x, factor_range=(-20, -20)),  # Brillo mínimo
    lambda x: random_brightness(x, factor_range=(20, 20))  # Brillo máximo
]
titles_brightness = [
    'Original',
    'Brillo Min',
    'Brillo Max'
]

transform_functions_contrast = [
    lambda x: x,  # Función de identidad para la imagen original
    lambda x: random_contrast(x, factor_range=(0.8, 0.8)),  # Contraste mínimo
    lambda x: random_contrast(x, factor_range=(1.2, 1.2))  # Contraste máximo
]
titles_contrast = [
    'Original',
    'Contraste Min',
    'Contraste Max'
]

transform_functions_saturation = [
    lambda x: x,  # Función de identidad para la imagen original
    lambda x: random_saturation(x, factor_range=(0.1, 0.1)),  # Saturación mínima
    lambda x: random_saturation(x, factor_range=(1.2, 1.2))  # Saturación máxima
]
titles_saturation = [
    'Original',
    'Saturación Min',
    'Saturación Max'
]

# Mostrar imágenes con transformaciones en gráficos separados para cada tipo de transformación
plot_images_with_transforms(image1, 'Imagen Original 1', transform_functions_brightness, titles_brightness)
plot_images_with_transforms(image2, 'Imagen Original 2', transform_functions_contrast, titles_contrast)
plot_images_with_transforms(image3, 'Imagen Original 3', transform_functions_saturation, titles_saturation)
